In [ ]:
from typing import Annotated, TypedDict, Literal
from langgraph.graph import StateGraph, START, END
from langgraph.prebuilt import ToolNode , tools_condition
from langgraph.graph.message import add_messages, AnyMessage
from langgraph.checkpoint.memory import MemorySaver
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.prompts import ChatPromptTemplate
from langchain.tools import tool
from IPython.display import display, Image
from dotenv import load_dotenv

import os

load_dotenv()  # Load environment variables from .env file
os.environ["GOOGLE_API_KEY"] = os.getenv("GOOGLE_API_KEY")

In [ ]:
class State(TypedDict):
    messages: Annotated[list[AnyMessage], add_messages]


def python_llm(requirements:str):
    """
    Generates Python code based on user requirements.
    Args:
        requirements (str): The user's requirements for the Python code.
    Returns:
        str: The generated Python code.
    """
    llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash")
    prompt = ChatPromptTemplate.from_messages({
        ("system", "You are a helpful assistant that writes Python code. return only the code for the requirements in the conversation, do not include any explanations."),
        ("user", "{requirements}")
    })
    chain = prompt | llm
    response = chain.invoke({ "requirements" : requirements })
    return response.content


def js_llm(requirements:str):
    """Generates Typescript code based on user requirements.
    Args:
        requirements (str): The user's requirements for the Typescript code.
    Returns:
        str: The generated Typescript code.
    """
    llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash")
    prompt = ChatPromptTemplate.from_messages({
        ("system", "You are a helpful assistant that writes Typescript code. return only the code for the requirements in the conversation, do not include any explanations."),
        ("user", "{requirements}")
    })
    chain = prompt | llm
    response = chain.invoke({ "requirements" : requirements })
    return response.content


def start_chat(state: State):
    llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash")
    tools = [python_llm, js_llm]
    llm_with_tools = llm.bind_tools(tools)
    
    return { "messages": llm_with_tools.invoke(state["messages"]) }

In [ ]:
tools = [python_llm, js_llm]

In [ ]:
graph = StateGraph(State)
graph.add_node("start_chat", start_chat)
graph.add_node("tools", ToolNode(tools))



graph.add_edge(START, "start_chat")
graph.add_conditional_edges("start_chat", tools_condition)
graph.add_edge("tools","start_chat")
graph.add_edge("start_chat",END)

memory = MemorySaver()
graph_builder = graph.compile(checkpointer=memory)


display(Image(graph_builder.get_graph().draw_mermaid_png()))

In [ ]:
config = { "configurable": {"thread_id": "2"}}

In [ ]:
results = graph_builder.invoke({
    "messages": [
        {"role": "user", "content": "I want to create a function that takes a list of numbers and returns the sum of all even numbers in the list in python"}
    ]
}, config)
for message in results["messages"]:
   message.pretty_print()

In [ ]:
follow_up = graph_builder.invoke({
    'messages': [
        {
            "role": "user" , "content" : "can you convert it into js or TS?"
        }
    ]
    
}, config)

for message in follow_up['messages']:
    message.pretty_print()

In [ ]:
follow_up = graph_builder.invoke({
    'messages': [
        {
            "role": "user" , "content" : "now can you write test in JS?"
        }
    ]
    
}, config)

for message in follow_up['messages']:
    message.pretty_print()